In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [2]:
# 5% duty cycle
log_1_5_duty_cycle = pd.read_csv('csv/log_1-5-duty-cycle.csv')
log_2_5_duty_cycle = pd.read_csv('csv/log_2-5-duty-cycle.csv')
log_3_5_duty_cycle = pd.read_csv('csv/log_3-5-duty-cycle.csv')
log_4_5_duty_cycle = pd.read_csv('csv/log_4-5-duty-cycle.csv')
log_5_5_duty_cycle = pd.read_csv('csv/log_5-5-duty-cycle.csv')
log_6_5_duty_cycle = pd.read_csv('csv/log_6-5-duty-cycle.csv')
log_7_5_duty_cycle = pd.read_csv('csv/log_7-5-duty-cycle.csv')
log_8_5_duty_cycle = pd.read_csv('csv/log_8-5-duty-cycle.csv')
log_9_5_duty_cycle = pd.read_csv('csv/log_9-5-duty-cycle.csv')
log_10_5_duty_cycle = pd.read_csv('csv/log_10-5-duty-cycle.csv')
log_11_5_duty_cycle = pd.read_csv('csv/log_11-5-duty-cycle.csv')

combined_5_duty_cycle = pd.concat([log_1_5_duty_cycle, log_2_5_duty_cycle, log_3_5_duty_cycle, log_4_5_duty_cycle, log_5_5_duty_cycle, log_6_5_duty_cycle, log_7_5_duty_cycle, log_8_5_duty_cycle, log_9_5_duty_cycle, log_10_5_duty_cycle, log_11_5_duty_cycle])

In [3]:
# 10% duty cycle
log_1_10_duty_cycle = pd.read_csv('csv/log_1-10-duty-cycle.csv')
log_2_10_duty_cycle = pd.read_csv('csv/log_2-10-duty-cycle.csv')
log_3_10_duty_cycle = pd.read_csv('csv/log_3-10-duty-cycle.csv')
log_4_10_duty_cycle = pd.read_csv('csv/log_4-10-duty-cycle.csv')
log_5_10_duty_cycle = pd.read_csv('csv/log_5-10-duty-cycle.csv')
log_6_10_duty_cycle = pd.read_csv('csv/log_6-10-duty-cycle.csv')
log_7_10_duty_cycle = pd.read_csv('csv/log_7-10-duty-cycle.csv')
log_8_10_duty_cycle = pd.read_csv('csv/log_8-10-duty-cycle.csv')
log_9_10_duty_cycle = pd.read_csv('csv/log_9-10-duty-cycle.csv')
log_10_10_duty_cycle = pd.read_csv('csv/log_10-10-duty-cycle.csv')
log_11_10_duty_cycle = pd.read_csv('csv/log_11-10-duty-cycle.csv')

combined_10_duty_cycle = pd.concat([log_1_10_duty_cycle, log_2_10_duty_cycle, log_3_10_duty_cycle, log_4_10_duty_cycle, log_5_10_duty_cycle, log_6_10_duty_cycle, log_7_10_duty_cycle, log_8_10_duty_cycle, log_9_10_duty_cycle, log_10_10_duty_cycle, log_11_10_duty_cycle])

In [4]:
# 20% duty cycle
log_1_20_duty_cycle = pd.read_csv('csv/log_1-20-duty-cycle.csv')
log_2_20_duty_cycle = pd.read_csv('csv/log_2-20-duty-cycle.csv')
log_3_20_duty_cycle = pd.read_csv('csv/log_3-20-duty-cycle.csv')
log_4_20_duty_cycle = pd.read_csv('csv/log_4-20-duty-cycle.csv')
log_5_20_duty_cycle = pd.read_csv('csv/log_5-20-duty-cycle.csv')
log_6_20_duty_cycle = pd.read_csv('csv/log_6-20-duty-cycle.csv')
log_7_20_duty_cycle = pd.read_csv('csv/log_7-20-duty-cycle.csv')
log_8_20_duty_cycle = pd.read_csv('csv/log_8-20-duty-cycle.csv')
log_9_20_duty_cycle = pd.read_csv('csv/log_9-20-duty-cycle.csv')
log_10_20_duty_cycle = pd.read_csv('csv/log_10-20-duty-cycle.csv')
log_11_20_duty_cycle = pd.read_csv('csv/log_11-20-duty-cycle.csv')

combined_20_duty_cycle = pd.concat([log_1_20_duty_cycle, log_2_20_duty_cycle, log_3_20_duty_cycle, log_4_20_duty_cycle, log_5_20_duty_cycle, log_6_20_duty_cycle, log_7_20_duty_cycle, log_8_20_duty_cycle, log_9_20_duty_cycle, log_10_20_duty_cycle, log_11_20_duty_cycle])

In [5]:
# 30% duty cycle
log_1_30_duty_cycle = pd.read_csv('csv/log_1-30-duty-cycle.csv')
log_2_30_duty_cycle = pd.read_csv('csv/log_2-30-duty-cycle.csv')
log_3_30_duty_cycle = pd.read_csv('csv/log_3-30-duty-cycle.csv')
log_4_30_duty_cycle = pd.read_csv('csv/log_4-30-duty-cycle.csv')
log_5_30_duty_cycle = pd.read_csv('csv/log_5-30-duty-cycle.csv')
log_6_30_duty_cycle = pd.read_csv('csv/log_6-30-duty-cycle.csv')
log_7_30_duty_cycle = pd.read_csv('csv/log_7-30-duty-cycle.csv')
log_8_30_duty_cycle = pd.read_csv('csv/log_8-30-duty-cycle.csv')
log_9_30_duty_cycle = pd.read_csv('csv/log_9-30-duty-cycle.csv')
log_10_30_duty_cycle = pd.read_csv('csv/log_10-30-duty-cycle.csv')
log_11_30_duty_cycle = pd.read_csv('csv/log_11-30-duty-cycle.csv')

combined_30_duty_cycle = pd.concat([log_1_30_duty_cycle, log_2_30_duty_cycle, log_3_30_duty_cycle, log_4_30_duty_cycle, log_5_30_duty_cycle, log_6_30_duty_cycle, log_7_30_duty_cycle, log_8_30_duty_cycle, log_9_30_duty_cycle, log_10_30_duty_cycle, log_11_30_duty_cycle])

In [6]:
# 40% duty cycle
log_1_40_duty_cycle = pd.read_csv('csv/log_1-40-duty-cycle.csv')
log_2_40_duty_cycle = pd.read_csv('csv/log_2-40-duty-cycle.csv')
log_3_40_duty_cycle = pd.read_csv('csv/log_3-40-duty-cycle.csv')
log_4_40_duty_cycle = pd.read_csv('csv/log_4-40-duty-cycle.csv')
log_5_40_duty_cycle = pd.read_csv('csv/log_5-40-duty-cycle.csv')
log_6_40_duty_cycle = pd.read_csv('csv/log_6-40-duty-cycle.csv')
log_7_40_duty_cycle = pd.read_csv('csv/log_7-40-duty-cycle.csv')
log_8_40_duty_cycle = pd.read_csv('csv/log_8-40-duty-cycle.csv')
log_9_40_duty_cycle = pd.read_csv('csv/log_9-40-duty-cycle.csv')
log_10_40_duty_cycle = pd.read_csv('csv/log_10-40-duty-cycle.csv')

combined_40_duty_cycle = pd.concat([log_1_40_duty_cycle, log_2_40_duty_cycle, log_3_40_duty_cycle, log_4_40_duty_cycle, log_5_40_duty_cycle, log_6_40_duty_cycle, log_7_40_duty_cycle, log_8_40_duty_cycle, log_9_40_duty_cycle, log_10_40_duty_cycle])

In [7]:
# 50% duty cycle
log_1_50_duty_cycle = pd.read_csv('csv/log_1-50-duty-cycle.csv')
log_2_50_duty_cycle = pd.read_csv('csv/log_2-50-duty-cycle.csv')
log_3_50_duty_cycle = pd.read_csv('csv/log_3-50-duty-cycle.csv')
log_4_50_duty_cycle = pd.read_csv('csv/log_4-50-duty-cycle.csv')
log_5_50_duty_cycle = pd.read_csv('csv/log_5-50-duty-cycle.csv')
log_6_50_duty_cycle = pd.read_csv('csv/log_6-50-duty-cycle.csv')
log_7_50_duty_cycle = pd.read_csv('csv/log_7-50-duty-cycle.csv')
log_8_50_duty_cycle = pd.read_csv('csv/log_8-50-duty-cycle.csv')
log_9_50_duty_cycle = pd.read_csv('csv/log_9-50-duty-cycle.csv')
log_10_50_duty_cycle = pd.read_csv('csv/log_10-50-duty-cycle.csv')
log_11_50_duty_cycle = pd.read_csv('csv/log_11-50-duty-cycle.csv')

combined_50_duty_cycle = pd.concat([log_1_50_duty_cycle, log_2_50_duty_cycle, log_3_50_duty_cycle, log_4_50_duty_cycle, log_5_50_duty_cycle, log_6_50_duty_cycle, log_7_50_duty_cycle, log_8_50_duty_cycle, log_9_50_duty_cycle, log_10_50_duty_cycle, log_11_50_duty_cycle])

In [9]:
# 60% duty cycle
log_1_60_duty_cycle = pd.read_csv('csv/log_1-60-duty-cycle.csv')
log_2_60_duty_cycle = pd.read_csv('csv/log_2-60-duty-cycle.csv')
log_3_60_duty_cycle = pd.read_csv('csv/log_3-60-duty-cycle.csv')
log_4_60_duty_cycle = pd.read_csv('csv/log_4-60-duty-cycle.csv')
log_5_60_duty_cycle = pd.read_csv('csv/log_5-60-duty-cycle.csv')
log_6_60_duty_cycle = pd.read_csv('csv/log_6-60-duty-cycle.csv')
log_7_60_duty_cycle = pd.read_csv('csv/log_7-60-duty-cycle.csv')
log_8_60_duty_cycle = pd.read_csv('csv/log_8-60-duty-cycle.csv')
log_9_60_duty_cycle = pd.read_csv('csv/log_9-60-duty-cycle.csv')
log_10_60_duty_cycle = pd.read_csv('csv/log_10-60-duty-cycle.csv')
log_11_60_duty_cycle = pd.read_csv('csv/log_11-60-duty-cycle.csv')

combined_60_duty_cycle = pd.concat([log_1_60_duty_cycle, log_2_60_duty_cycle, log_3_60_duty_cycle, log_4_60_duty_cycle, log_5_60_duty_cycle, log_6_60_duty_cycle, log_7_60_duty_cycle, log_8_60_duty_cycle, log_9_60_duty_cycle, log_10_60_duty_cycle, log_11_60_duty_cycle])

In [10]:
# For 95% confidence interval
z = 2.228

In [11]:
def plot_error_bar(mean, margin_of_error, title, x_label, y_label):
  plt.errorbar(0, mean, yerr=margin_of_error, fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0)
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()

In [ ]:
def calculate_mean_and_margin_of_error(data):
  mean = data.mean()
  std = data.std()
  margin_of_error = z * (std / np.sqrt(n))
  return mean, margin_of_error

In [ ]:
mean_5_duty_cycle, margin_of_error_5_duty_cycle = calculate_mean_and_margin_of_error(combined_5_duty_cycle['_Eihop'])
mean_10_duty_cycle, margin_of_error_10_duty_cycle = calculate_mean_and_margin_of_error(combined_10_duty_cycle['_Eihop'])
mean_20_duty_cycle, margin_of_error_20_duty_cycle = calculate_mean_and_margin_of_error(combined_20_duty_cycle['_Eihop'])
mean_30_duty_cycle, margin_of_error_30_duty_cycle = calculate_mean_and_margin_of_error(combined_30_duty_cycle['_Eihop'])
mean_40_duty_cycle, margin_of_error_40_duty_cycle = calculate_mean_and_margin_of_error(combined_40_duty_cycle['_Eihop'])
mean_50_duty_cycle, margin_of_error_50_duty_cycle = calculate_mean_and_margin_of_error(combined_50_duty_cycle['_Eihop'])
mean_60_duty_cycle, margin_of_error_60_duty_cycle = calculate_mean_and_margin_of_error(combined_60_duty_cycle['_Eihop'])
